In [1]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
plt.rcParams['image.cmap'] = 'gist_earth'
np.random.seed(98765)

In [ ]:
# Data Augmentation 
# Random Crop
# Random Flip
import os
import skimage
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from torchvision.transforms import Compose, TenCrop, Lambda
import torchvision.transforms as vis_tf
from glob import glob
import cv2
from matplotlib import pyplot as plt
%matplotlib inline
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('/Users/trimchala/BreastPathQ/breastpathq/tf_unet/')
import tf_unet
from tf_unet.image_util import BaseDataProvider

%load_ext autoreload

%autoreload 2
plt.ion()   # interactive mode

dataset_path = '/Users/trimchala/BreastPathQ/breastpathq/datasets/MoNuSegTraining'
image_path = os.path.join(dataset_path, 'TissueImages')
mask_path = os.path.join(dataset_path, 'LabeledNucImages')


class MonuSegDataProvider(BaseDataProvider):
    def __init__(
        self, 
        image_path, 
        mask_path, 
        transforms=None, shuffle_data=True, 
        a_min=None, a_max=None, 
    ):
        super(MonuSegDataProvider, self).__init__(a_min, a_max)
        # stuff
        self.image_path = image_path
        self.mask_path = mask_path
        self.file_idx = -1
        self.transforms = transforms
        self.image_list = glob(os.path.join(self.image_path, "*.tif"))
        self.mask_list = glob(os.path.join(self.mask_path, "*.png"))
        self.shuffle_data = shuffle_data
        
#         if self.shuffle_data:
#             np.random.shuffle(self.data_files)
        
        assert len(self.image_list) > 0, "No training files"
        print("Number of files used: %s" % len(self.image_list))
        
        test_img = cv2.imread(image_list[0])
        self.channels = 1 if len(test_img.shape) == 2 else test_img.shape[-1]
        
    def _load_file(self, path, dtype=np.float32):
        return np.array(PIL.Image.open(path), dtype)

    
    def _cylce_file(self):
        self.file_idx += 1
        if self.file_idx >= len(self.image_list):
            self.file_idx = 0 
#             if self.shuffle_data:
#                 np.random.shuffle(self.image_list)


    def _next_data(self):
        self._cylce_file()
        image_file = self.image_list[self.file_idx]
        mask_file = self.mask_list[self.file_idx]
        
        img = cv2.imread(image_file)        
        mask_gray = np.array(
            PIL.Image.fromarray(cv2.imread(mask_list[0])).convert('L')
        )
#         mask0 = (mask_gray <= mask_gray.min()).astype(int)
#         mask1 = (mask_gray > mask_gray.min()).astype(int)
#         mask = np.dstack([mask0, mask1])
        mask = (mask_gray <= mask_gray.min()).astype(int)
                        
#         if self.transforms is not None:
#             img = self.transforms(img)
#             mask = self.transforms(mask)
#         else: 
#             pil2tensor = vis_tf.Compose([vis_tf.ToTensor()])
#             img = pil2tensor(img)
#             mask = pil2tensor(mask)
    
        return img, mask


In [144]:
import os
import sys
sys.path.append('/Users/trimchala/BreastPathQ/breastpathq/tf_unet/')
sys.path.append('/Users/trimchala/BreastPathQ/breastpathq/tf_unet/tf_unet/')
sys.path.append('/Users/trimchala/BreastPathQ/breastpathq/tf_unet/scripts/')
os.chdir('/Users/trimchala/BreastPathQ/breastpathq/tf_unet/')
from tf_unet import image_gen
from tf_unet import unet
from tf_unet import util

monuseg_generator = MonuSegDataProvider(image_path, mask_path)

unet32 = unet.Unet(
    channels=monuseg_generator.channels, 
    n_class=monuseg_generator.n_class, 
    layers=3, 
    features_root=16
)

trainer = unet.Trainer(unet32, optimizer="momentum", opt_kwargs=dict(momentum=0.2))

2018-11-23 17:36:43,761 Layers 3, features 16, filter size 3x3, pool size: 2x2


Number of files used: 30


In [145]:
image, mask = monuseg_generator._next_data()
image.shape, mask.shape

((1000, 1000, 3), (1000, 1000))

In [ ]:
path = trainer.train(
    monuseg_generator, 
    "./tf_unet_monuseg_trained", 
    training_iters=32, epochs=10,
    display_step=5
)

2018-11-23 17:36:50,104 Removing '/Users/trimchala/BreastPathQ/breastpathq/tf_unet/prediction'
2018-11-23 17:36:50,106 Removing '/Users/trimchala/BreastPathQ/breastpathq/tf_unet/tf_unet_monuseg_trained'
2018-11-23 17:36:50,108 Allocating '/Users/trimchala/BreastPathQ/breastpathq/tf_unet/prediction'
2018-11-23 17:36:50,109 Allocating '/Users/trimchala/BreastPathQ/breastpathq/tf_unet/tf_unet_monuseg_trained'
2018-11-23 17:37:26,533 Verification error= 0.0%, loss= -0.8830
2018-11-23 17:37:29,105 Start optimization
2018-11-23 17:37:55,206 Iter 0, Minibatch Loss= -1.1550, Training Accuracy= 1.0000, Minibatch error= 0.0%
2018-11-23 17:39:01,935 Iter 5, Minibatch Loss= -2.3860, Training Accuracy= 1.0000, Minibatch error= 0.0%
2018-11-23 17:39:58,143 Iter 10, Minibatch Loss= -2.3684, Training Accuracy= 1.0000, Minibatch error= 0.0%
2018-11-23 17:40:48,996 Iter 15, Minibatch Loss= -2.4253, Training Accuracy= 1.0000, Minibatch error= 0.0%
2018-11-23 17:42:05,376 Iter 20, Minibatch Loss= -2.4367,

## Stepping through unet.Trainier